In [63]:
import numpy as np

# Get OTM output from Python

In [80]:
list_link_ids = ['27','28','29']
list_ramp_ids = ['511','911'] # inlink for '27','29'
den_none      = 0
max_ramp_flow = 800
tolerance     = 10


# otm.run()

# Version A (when we have OTM functions):

# list_link_den     = otm.get_density(list_link_ids)
# list_link_den_cri = otm.get_cri_density(list_link_ids)
# list_link_den_jam = otm.get_jam_density(list_link_ids)
# list_ramp_speed   = otm.get_speed(list_ramp_ids)

# Version B (when we don't have OTM functions):

list_link_den     = [56,77,94]
list_link_den_cri = [60,60,60]
list_link_den_jam = [120,120,120]
list_ramp_speed   = [80,80]


# Initialize Q-table with all zeros

In [65]:
possible_den = np.arange(den_none, max(list_link_den_jam), 20)
possible_act = [0,1] # 0: full red light; 1: full green light

state_space = np.arange( 0, len(possible_den)**len(list_link_ids), 1)
act_space   = np.arange( 0, len(possible_act)**len(list_ramp_ids), 1)

print(state_space)
print(act_space)

Q    = np.zeros([len(state_space),len(act_space)])
Qnew = np.ones( [len(state_space),len(act_space)])

print(Q.shape)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215]
[0 1 2 3]
(216, 4)


In [81]:
def enumer_A(list1, list2):
    list_comb = []
    i = 0
    for x in list1:
        for y in list1:
            list_comb.append((i,x,y))
            i += 1
    return list_comb

In [82]:
def enumer_B(list1, list2):
    list_comb = []
    i = 0
    for x in list1:
        for y in list1:
            for z in list1:
                list_comb.append((i,x,y,z))
                i += 1
    return list_comb

In [67]:
def reward(list1, list2, tolerance):
    for i in range(len(list1)):
        if abs( list1[i] - list2[i] ) <= tolerance:
            r = 1
        elif list1[i] - list2[i] <= 0:
            r = 0.5
        else:
            r = -1
        list_r.append(r)
    
    return min(list_r)

In [68]:
# Set learning parameters
learning = 0.8
discount = 0.95
num_episodes = 2000
max_gap = 5

In [69]:
def quantize(list1, list2):
    global i_q
    for i in list1:
        for j in list2:
            if j-10 <= i < j:
                i_q = j
        list_q.append(i_q)    
    return list_q

In [95]:
list_q = []
state  = enumer_B( possible_den , list_link_ids )
list_link_den_qnt  = quantize(list_link_den, possible_den)
print(state) 
print(list_link_den_qnt) # it is the 137th item in the list
loc = [i for i, x, y, z in state if list((x, y, z)) == list_link_den_qnt]
print(loc[0])

[60, 80, 100]
137


In [101]:
list_q = []
action = enumer_A( possible_act , list_ramp_ids )
print(action) 
print(list_ramp_flow) # it is the th item in the list
loc = [i for i, x, y, z in state if list((x, y, z)) == list_ramp_flow]
print(loc[0])

[(0, 0, 0), (1, 0, 1), (2, 1, 0), (3, 1, 1)]
[800, 800, 800, 800]


IndexError: list index out of range

In [98]:
total_r = []
for i in range(num_episodes):
    
    # Reset the environment
    s     = 1
    r_sum = 0
    j     = 0
    
    while j < 10:
                
        j += 1
        noise = 0
        #noise = np.random.randn(1,len(act_space))*(1./(i+1))
        
        
        # 1. OTM running and get the new info
        
        # Version A (when we have OTM functions):

        # list_link_den     = otm.get_density(list_link_ids)
        # list_link_den_cri = otm.get_cri_density(list_link_ids)
        # list_ramp_speed   = otm.get_speed(list_ramp_ids)

        # Version B (when we don't have OTM functions):

        list_link_den     = [56,77,94]
        list_link_den_cri = [60,60,60]
        list_ramp_speed   = [80,80]
        
        
        list_ramp_flow    = max_ramp_flow / list_ramp_speed

        
        action = enumer_A( possible_act , list_ramp_ids )
        state  = enumer_B( possible_den , list_link_ids )
        
        list_q = []
        list_r = []
        
        list_link_den_qnt  = quantize(list_link_den, possible_den)
        
        
        loc_a = [i for i, x, y    in action if list((x, y   )) == list_ramp_flow]
        loc_s = [i for i, x, y, z in state  if list((x, y, z)) == list_link_den_qnt]
        
        a  = np.argmax(Q[s,:] + noise)
        a1 = loc_a[0]
        s1 = loc_s[0]
        r  = reward(list_link_den_qnt, list_link_den_cri, tolerance)
           
        
        # 2. Q-table iteration
                              
        Q[s,a] = Q[s,a] + learning * (r + discount * np.max( Q[s1,:] ) - Q[s,a] )
        
        r_sum += r
        s = s1
        
        # 3. Break if gap is too small
        
        gap = abs(Qnew[s,a] - Q[s,a])
        
        if gap < max_gap:
            break
        
        Qnew[s,a] = Q[s,a]
        
    total_r.append(r_sum)

In [99]:
print(Q) # continuous, not differentiable (symmetric)

[[ 0.  0.  0.  0.]
 [-1. -1. -1. -1.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0

In [100]:
print(total_r)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,